In [1]:
import pandas as pd
import pyodbc
import json
from pathlib import Path

In [2]:
connection = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                            'Server=mde-dw1-uat;'
                            'Database=MDE_DM_ESSA_REPORT_CARD;'
                            'Trusted_Connection=yes;')
schoolYear = 2017

In [3]:
entitiesQuery = '''
SELECT
    EntityId,
    EntitySchoolYear,
    EntityDisplayName,
    EntitySearchName,
    EntityType,
    EntityGrade,
    EntityAccountabilityScore,
    EntityAddress,
    EntityCity,
    EntityState,
    EntityZip,
    EntityContactName,
    EntityContactEmail,
    EntityLevel,
    ParentEntityName,
    (SELECT P.EntityId FROM [MDE_DM_ESSA_REPORT_CARD].[dbo].[ReportCardEntity] P where C.ParentEntityRowId = P.EntityRowId) AS ParentEntityId,
    EntityTSICSI,
    EntityTSICSIDesc,
    EntityGradeWithEL,
    EntityNCESId,
    EntityLatitude,
    EntityLongitude
FROM MDE_DM_ESSA_REPORT_CARD.dbo.ReportCardEntity C
WHERE EntitySchoolYear = 2017
'''

In [4]:
dfEntities = pd.read_sql(entitiesQuery, connection)

In [5]:
dfEntities.head(3)
output_file = 'entities.json'
output_dir = Path('../2017/entities/')
output_dir.mkdir(parents=True, exist_ok=True)
dfEntities.to_json(output_dir / output_file, orient='records')

In [6]:
factsQuery = '''
SELECT
    tgt.ReportCardYear,
    e.EntityId,
    ag.AggregationCode,
    ag.AggregationName,
    ag.AggregationSortOrder,
    ag.AggregationLevelCategoryRowId,
    ag.AggregationMSISAMOGroupId,
    ag.AggregationEDFactsGroup,
    agc.AggregationLevelCategoryCode,
    agc.AggregationLevelCategoryName,
    agc.AggregationLevelCategoryShortDesc,
    agc.AggregationLevelCategoryLongDesc,
    agc.AggregationLevelCategorySortOrder,
    c.ComponentCode,
    c.ComponentName,
    c.ComponentShortDesc,
    c.ComponentLongDesc,
    c.ComponentNoDataMsg,
    c.ComponentTypeRowId,
    c.ComponentGroupRowId,
    c.ComponentEntityLevel,
    c.ComponentDetailsPage,
    c.ComponentIconFlag,
    c.ComponentSortOrder,
    c.ComponentDisplayFormat,
    c.ParentComponentRowId,
    c.ComponentHiddenOnTilePage,
    c.ComponentIconURL,
    tgt.ComponentLevel,
    tgt.ComponentValue,
    ct.ComponentTypeCode,
    ct.ComponentTypeName,
    ct.ComponentTypeShortDesc,
    ct.ComponentTypeLongDesc,
    ct.ComponentGroupRowId,
    ct.ComponentTypeSortOrder,
    cg.ComponentGroupCode,
    cg.ComponentGroupName,
    cg.ComponentGroupShortDesc,
    cg.ComponentGroupLongDesc,
    cg.ComponentGroupSortOrder,
    cgl.InterimGoal,
    cgl.LongTermGoal,
    cgl.IsActive,
    cgl.InterimGoal2,
    cgl.InterimGoalLabel,
    cgl.InterimGoalLabel2,
    cgl.LongTermGoalLabel,
    sl.SuppressionLevelCode,
    sl.SuppressionLevelName,
    sl.SuppressionLevelDescription,
    st.SuppressionTextCode,
    st.SuppressionLevelShortDesc,
    st.SuppressionLevelLongDesc,
    ntl.SchoolYear AS NSchoolYear,
    ntl.ComponentCode AS NComponentCode,
    ntl.AggregationCode AS NAggregationCode,
    ntl.N,
    ntl.NofN,
    ntl.NLowerValue,
    ntl.NUpperValue,
    ntl.NDisplayLabel,
    ntl.NDescription,
    ntl.EntityType AS NEntityType,
    ntl.IS_ACTIVE AS NIS_ACTIVE,
    ntl.ComponentPercentile AS NComponentPercentile
FROM [dbo].[ESSAReportCardFact] tgt
    JOIN [dbo].ReportCardEntity e ON e.EntityRowId = tgt.EntityRowId
    JOIN [dbo].AggregationLevel ag ON  ag.AggregationRowId = tgt.AggregationLevelRowId
    LEFT JOIN [dbo].AggregationLevelCategory agc ON agc.AggregationLevelCategoryRowId = tgt.AggregationLevelCategoryRowId
    JOIN [dbo].component c on  c.ComponentRowId = tgt.ComponentRowId
    JOIN [dbo].ComponentType ct on ct.ComponentTypeRowId =tgt.ComponentTypeRowId
    JOIN [dbo].ComponentGroup cg on cg.ComponentGroupRowId = tgt.ComponentGroupRowId
    LEFT JOIN [dbo].ComponentGoal cgl on cgl.ComponentGoalRowId = tgt.ComponentGoalRowId
    LEFT JOIN [dbo].SuppressionLevel sl on sl.SuppressionLevelRowID = tgt.ComponentSuppressionLevelRowId
    LEFT JOIN [dbo].SuppressionText st on st.SuppressionTextRowID = tgt.ComponentSuppressionTextRowId
    LEFT JOIN [dbo].ComponentNTile ntl on ntl.ComponentNTileRowId = tgt.ComponentNTileRowId
WHERE tgt.ReportCardYear = 2017
'''

In [7]:
dfFacts = pd.read_sql(factsQuery, connection)

In [8]:
dfFacts.head(3)

ReportCardYear  EntityId AggregationCode AggregationName  \
0            2017  0900-004             ALL             All   
1            2017  0920-004             ALL             All   
2            2017  2400-016             ALL             All   

   AggregationSortOrder  AggregationLevelCategoryRowId  \
0                     1                              1   
1                     1                              1   
2                     1                              1   

   AggregationMSISAMOGroupId AggregationEDFactsGroup  \
0                          1                     N/A   
1                          1                     N/A   
2                          1                     N/A   

  AggregationLevelCategoryCode AggregationLevelCategoryName  ...  \
0                          OVW                     Overview  ...   
1                          OVW                     Overview  ...   
2                          OVW                     Overview  ...   

  NAggregationCode   N  NofN NLowerValue NUpperValue NDisplayLabel  \
0             None NaN   NaN         NaN         NaN          None   
1             None NaN   NaN         NaN         NaN          None   
2             None NaN   NaN         NaN         NaN          None   

  NDescription NEntityType  NIS_ACTIVE  NComponentPercentile  
0         None        None        None                   NaN  
1         None        None        None                   NaN  
2         None        None        None                   NaN  

[3 rows x 66 columns]

In [9]:
dfFactsGrouped = (
    dfFacts.groupby([
        'EntityId','AggregationCode','AggregationName'
    ]).apply(lambda x: x[[
        'AggregationCode',
        'AggregationName',
        'AggregationSortOrder',
    'AggregationLevelCategoryRowId',
    'AggregationMSISAMOGroupId',
    'AggregationEDFactsGroup',
    'AggregationLevelCategoryCode',
    'AggregationLevelCategoryName',
    'AggregationLevelCategoryShortDesc',
    'AggregationLevelCategoryLongDesc',
    'AggregationLevelCategorySortOrder',
    'ComponentCode',
    'ComponentName',
    'ComponentShortDesc',
    'ComponentLongDesc',
    'ComponentNoDataMsg',
    'ComponentTypeRowId',
    'ComponentGroupRowId',
    'ComponentEntityLevel',
    'ComponentDetailsPage',
    'ComponentIconFlag',
    'ComponentSortOrder',
    'ComponentDisplayFormat',
    'ParentComponentRowId',
    'ComponentHiddenOnTilePage',
    'ComponentIconURL',
    'ComponentLevel',
    'ComponentValue',
    'ComponentTypeCode',
    'ComponentTypeName',
    'ComponentTypeShortDesc',
    'ComponentTypeLongDesc',
    'ComponentGroupRowId',
    'ComponentTypeSortOrder',
    'ComponentGroupCode',
    'ComponentGroupName',
    'ComponentGroupShortDesc',
    'ComponentGroupLongDesc',
    'ComponentGroupSortOrder',
    'InterimGoal',
    'LongTermGoal',
    'IsActive',
    'InterimGoal2',
    'InterimGoalLabel',
    'InterimGoalLabel2',
    'LongTermGoalLabel',
    'SuppressionLevelCode',
    'SuppressionLevelName',
    'SuppressionLevelDescription',
    'SuppressionTextCode',
    'SuppressionLevelShortDesc',
    'SuppressionLevelLongDesc',
    'NSchoolYear',
    'NComponentCode',
    'NAggregationCode',
    'N',
    'NofN',
    'NLowerValue',
    'NUpperValue',
    'NDisplayLabel',
    'NDescription',
    'NEntityType',
    'NIS_ACTIVE',
    'NComponentPercentile'
    ]].to_dict('records'))
     .reset_index()
     .rename(columns={0: 'Data'}))

/usr/local/Caskroom/miniconda/base/envs/serverless/lib/python3.7/site-packages/ipykernel_launcher.py:69: UserWarning: DataFrame columns are not unique, some columns will be omitted.


In [10]:
dfFactsGrouped.head(3)
dfFactsGrouped = dfFactsGrouped.groupby(['EntityId', 'AggregationCode'])

In [11]:
for i, item in dfFactsGrouped:
    output_file = i[1] + '.json'
    output_dir = Path('../2017/facts/' + i[0])
    output_dir.mkdir(parents=True, exist_ok=True)
    item.to_json(output_dir / output_file, orient='records')